In [186]:
import pandas as pd
import json
import collections
import scipy.io
import numpy as np
import mpu
import math

In [218]:
bus = scipy.io.loadmat('MATPOWER/bus.mat')['bus']
power_demand = bus[:, 2]
df = pd.read_csv("GIS/CATS_gens.csv")
df = df[df['Pmax'] != 0.0].to_numpy()
#df = df.drop_duplicates(subset=['PlantCode','GenID']).to_numpy()

In [219]:
gen_emission = pd.read_excel("emissions2022.xlsx")
gen_emission = gen_emission[gen_emission['Generation (kWh)'] > 0]
gen_emission = gen_emission[gen_emission['State'] == 'CA']
gen_emission = gen_emission[['Plant Code','Fuel Code','Aggregated Fuel Group','carbon emission rate']]
gen_emission = gen_emission.rename(columns={'Plant Code': 'Plant_Code'})
gen_emission.groupby(['Fuel Code'])['carbon emission rate'].mean()
# get carbon emssion rate of each fuel type (tons/mWh)

Fuel Code
BIT    4.702783
DFO    1.191170
GEO    0.028590
JF     0.988836
MSW    1.004947
NG     0.603798
PG     1.331544
WO     0.400283
Name: carbon emission rate, dtype: float64

In [220]:
type_to_emission = collections.defaultdict(float)
type_to_emission['Conventional Hydroelectric'] = 0.001
type_to_emission['Hydroelectric Pumped Storage'] = 0.001
type_to_emission['Petroleum Liquids'] = 1.191
type_to_emission['Natural Gas Internal Combustion Engine'] = 0.604
type_to_emission['Natural Gas Fired Combined Cycle'] = 0.604
type_to_emission['Natural Gas Steam Turbine'] = 0.604
type_to_emission['Natural Gas Fired Combustion Turbine'] = 0.604
#type_to_emission['Nuclear'] = 0
type_to_emission['Geothermal'] = 0.029
#type_to_emission['Onshore Wind Turbine'] = 0
type_to_emission['Other Waste Biomass'] = 0.1
type_to_emission['Wood/Wood Waste Biomass'] = 0.1
type_to_emission['Landfill Gas'] = 0.604
type_to_emission['Solar Photovoltaic'] = 0.001
type_to_emission['Solar Thermal without Energy Storage'] = 0.001
type_to_emission['Conventional Steam Coal'] = 4.703
type_to_emission['Other Gases'] = 0.8
type_to_emission['Batteries'] = 0.001
#type_to_emission['Petroleum Coke'] = 1808.5
type_to_emission['Municipal Solid Waste'] = 1.005
type_to_emission['Other Natural Gas'] = 0.604
type_to_emission['All Other'] = 0.1
#type_to_emission['IMPORT'] = 884
#type_to_emission['Synchronous Condenser'] = 884

In [221]:
branch_ = scipy.io.loadmat('MATPOWER/branch2.mat')['B']
branch_from_bus = list(map(int, branch_[:, 0]-1))
branch_to_bus = list(map(int, branch_[:, 1]-1))
line_to_nodes = [list( map(int,i) ) for i in branch_[:, 0:2]-1]


In [222]:
f = open("pf_solution.json")
sol = json.load(f)
gen = [0]*2149
gen_cost = [0]*2149
power_generation = [0]*2149
carbon_emission = [0]*2149
branch_power_to = [0]*10574
branch_power_from = [0]*10574
for line, val in sol['solution']['gen'].items():
    if val['pg'] != 0.0:
        gen[int(line)-1] = df[int(line)-1][2]-1
        carbon_emission[int(line)-1] = type_to_emission[df[int(line)-1][3]]
        gen_cost[int(line)-1] = val['pg_cost']*100
        power_generation[int(line)-1] = val['pg']*100

for line, val in sol['solution']['branch'].items():
    branch_power_from[int(line)-1] = val['pf']*100
    branch_power_to[int(line)-1] = val['pt']*100

f.close()

In [223]:
graph = collections.defaultdict(list) # from: (to, line)
graph_reverse = collections.defaultdict(list) # to: (from, line)
for i, (from_bus, to_bus) in enumerate(line_to_nodes):
    graph[from_bus].append((to_bus, i))
    graph_reverse[to_bus].append((from_bus, i))
for i, f in enumerate(branch_power_from):
    if f < 0.0:
        from_node, to_node = line_to_nodes[i]
        graph[from_node].remove((to_node, i))
        graph[to_node].append((from_node, i))
        graph_reverse[to_node].remove((from_node, i))
        graph_reverse[from_node].append((to_node, i))
        branch_power_from[i] = -f

In [224]:
num_bus = len(bus)
num_branch = len(branch_from_bus)
num_gen = len(gen)
print("Number of transmission lines and transformers: " + str(num_branch))
print("Number of buses: " + str(num_bus))
print("Number of generators: " + str(num_gen))

Number of transmission lines and transformers: 10574
Number of buses: 8870
Number of generators: 2149


In [226]:
# find cycles
def dfs_cycle(graph, n, visiting, curPath, start, visited):
    if visiting[n]:
        #if n == start:
        to_print = []
        for i in range(curPath.index(n), len(curPath)):
            to_print.append(curPath[i])
            visited[curPath[i]] = True
        cycles.add(frozenset(to_print))
        return
    visiting[n] = True
    curPath.append(n)
    for nei, _ in graph[n]:
        dfs_cycle(graph, nei, visiting, curPath, start, visited)
    visiting[n] = False
    curPath.pop(-1)

In [215]:
# find number of components in graph
visited = set()
num_components = 0
def find_components(graph, n, visited):
    if n in visited: return
    visited.add(n)
    for nei, _ in graph[n]:
        find_components(graph, nei, visited)
for n in range(num_bus):
    if n not in visited:
        find_components(graph, n, visited)
        num_components += 1
num_components
# we get num_compoents = 770

770

In [ ]:
#cycle start with: 2148 4471 4472
#cycle start with: 1904 3333 3334 3332
#start with: 2646 2647 2655
# start with: 2880 3394 3395
# start with: 3377 8267 3378
# start with: 4866 4868 4867

In [90]:
visited = [False]*num_bus
for n in range(num_bus):
    if not visited[n]:
        start = n
        visiting = [False]*num_bus
        dfs_cycle(graph, n, visiting, [], start, visited)

In [228]:
cycles = set()
cycles.add(frozenset((2148,4471,4472)))
cycles.add(frozenset((1904,3333,3334,3332)))
cycles.add(frozenset((2646,2647,2655)))
cycles.add(frozenset((2880,3394,3395)))
cycles.add(frozenset((3377,8267,3378)))
cycles.add(frozenset((4866,4868,4867)))

In [229]:
def update_graph_DAG(graph, c):
    for b in c:
        for nei, line in graph[b]:
            if nei in c:
                branch_power_from[line] = 0
                if (nei, line) in graph[b]:
                    graph[b].remove((nei, line))
        for nei, line in graph_reverse[b]:
            if nei in c:
                branch_power_from[line] = 0
                if (b, line) in graph[nei]:
                    graph[nei].remove((b, line))
        if b != c[0]:
            for nei, line in graph[b]:
                if nei not in c:
                    graph[c[0]].append((nei, line))
                    if (nei, line) in graph[b]:
                        graph[b].remove((nei, line))
            for nei, line in graph_reverse[b]:
                if nei not in c:
                    graph[nei].append((c[0], line))
                    if (b, line) in graph[nei]:
                        graph[nei].remove((b, line))
            graph[b] = []
    return graph

In [230]:
for c in cycles:
    c = list(c)
    total_demand = 0
    total_gen = 0
    for b in c:
        total_demand += power_demand[b]
        idx = gen.index(b) if b in gen else -1
        if idx != -1:
            total_gen += power_generation[idx]
    power_demand[c[0]] = total_demand
    idx = gen.index(b) if b in gen else -1
    if idx != -1:
        power_generation[gen.index(c[0])] = total_gen
    graph = update_graph_DAG(graph, c)


In [231]:
graph_reverse = collections.defaultdict(list)
for k, v in graph.items():
    for b, line in v:
        graph_reverse[b].append((k, line))

In [317]:
# Tarjan's Strongly Connected Component (SCC) Algorithm
UNVISITED = -1
id = [0]
sccCount = [0]
ids = [0]*num_bus
low = [0]*num_bus
onStack = [False]*num_bus
stack = []
def findSccs():
    for i in range(num_bus): ids[i] = UNVISITED
    for i in range(num_bus):
        if ids[i] == UNVISITED:
            tarjan_dfs(i)
    return low
def tarjan_dfs(at):
    stack.append(at)
    onStack[at] = True
    ids[at] = id[0]
    low[at] = id[0]
    id[0] += 1
    for nei, _ in graph[at]:
        if ids[nei] == UNVISITED:
            tarjan_dfs(nei)
        if onStack[nei]:
            low[at] = min(low[nei], low[at])
    if ids[at] == low[at]:
        while stack:
            node = stack.pop(-1)
            onStack[node] = False
            low[node] = ids[at]
            if node == at: break
        sccCount[0] += 1

In [232]:
# Kahn's algo
def topo_order_kahn(graph):
    in_degree = [0]*num_bus
    for i in range(num_bus):
        for nei, _ in graph[i]:
            in_degree[nei] += 1
    q = []
    for i in range(num_bus):
        if in_degree[i] == 0:
            q.append(i)
    index = 0
    order = [0]*num_bus
    while q:
        at = q.pop(0)
        order[index] = at
        index += 1
        for nei, _ in graph[at]:
            in_degree[nei] -= 1
            if in_degree[nei] == 0:
                q.append(nei)
    return in_degree

In [235]:
line_to_gen = collections.defaultdict(set)
node_to_gen = collections.defaultdict(set)
def dfs(g, n, visited):
    if n in visited: return
    visited.add(n)
    node_to_gen[n].add(g)
    for nei, line in graph[n]:
        line_to_gen[line].add(g)
        dfs(g, nei, visited)
for i, g in enumerate(gen):
    visited = set()
    dfs(g, g, visited)
line_prop_mat=np.zeros((num_gen, num_branch), dtype=float)
bus_prop_mat=np.zeros((num_gen, num_bus), dtype=float)
# initially, if node_to_gen only has 1 generator, bus_prop_mat start with 1
for k, v in node_to_gen.items():
    if len(v) == 1:
        idx = list(gen).index(list(v)[0])
        bus_prop_mat[idx][int(k)] = 1.0
for k, v in line_to_gen.items():
    if len(v) == 1:
        idx = list(gen).index(list(v)[0])
        line_prop_mat[idx][int(k)] = 1.0
# visit nodes in topological order
# step 1: calculate bus_prop based on in flowing lines_prop
# step 2: calculate line_prop of out flowing lines
in_degree = collections.defaultdict(int)
for i, v in graph_reverse.items():
    in_degree[i] = len(v)
q = [] # list of nodes with no inflow
topo_order = []
for g in gen:
    if in_degree[g] == 0:
        q.append(g)
while q:
    cur = int(q.pop(0))
    if len(node_to_gen[cur]) > 1:
        out_total = power_demand[cur]
        for nei, out_line in graph[cur]:
            out_total += branch_power_from[out_line]
        
        for g in node_to_gen[cur]:
            idx = list(gen).index(g)
            if cur == g:
                if out_total > 0.000001: 
                        #print("Power gen: " + str(power_generation[idx]))
                    bus_prop_mat[idx][cur] = power_generation[idx]/out_total
            else:
                for nei, in_line in graph_reverse[cur]:
                    if out_total > 0.000001: 
                        bus_prop_mat[idx][cur] += branch_power_from[in_line]*line_prop_mat[idx][in_line]/out_total
        for g in node_to_gen[cur]:
            idx = list(gen).index(g)
            for nei, out_line in graph[cur]:
                line_prop_mat[idx][out_line] = bus_prop_mat[idx][cur]

    topo_order.append(cur)
    for nei, line in graph[cur]:
        in_degree[nei] -= 1
        if in_degree[nei] == 0:
            q.append(nei)
carbon_vec = np.zeros((num_bus, 1), dtype=float)
bus_prop_mat_prop = np.zeros((num_gen, num_bus), dtype=float)
for i in range(num_gen):
    s = np.sum(bus_prop_mat[i])
    for j in range(num_bus):
        if s != 0.0:
            bus_prop_mat_prop[i][j] = bus_prop_mat[i][j]/s

for i in range(num_bus):
    for j in range(num_gen):
        #load[i] += bus_prop_mat[j,i] * power_generation.value[j]
        carbon_vec[i] += bus_prop_mat_prop[j,i] * power_generation[j] * carbon_emission[j]
total_cost = np.dot(gen_cost, power_generation)
total_carbon = sum(carbon_vec)


In [236]:
total_carbon

array([3577.92792852])

In [251]:
# in_degree_list = [0]*num_bus
# for k, v in in_degree.items():
#     in_degree_list[int(k)] = v
# def dfs_cycle(n, visited, total_gen, temp, in_degree_list):
#     if n in visited: return
#     visited.add(n)
#     if n in gen:
#         total_gen[0] += power_generation[gen.index(n)]
#         temp[n] = power_generation[gen.index(n)]
#     for nei, _ in graph[n]:
#         if in_degree_list[int(nei)] > 0:
#             dfs_cycle(int(nei), visited, total_gen, temp, in_degree_list)
#             in_degree_list[int(nei)] -= 1

In [185]:
# for i in range(len(in_degree_list)):
#     if in_degree_list[i] > 0 and i in gen and power_generation[gen.index(i)] > 0.000001 and len(graph[i]) != 0:
#         total_gen = [0.0]
#         visited = set()
#         temp = collections.defaultdict(float)
#         dfs_cycle(i, visited, total_gen, temp, in_degree_list)
#         if total_gen != 0:
#             for k, v in temp.items():
#                 print(v/total_gen[0])
#                 bus_prop_mat[gen.index(k)][k] = v/total_gen[0]


In [253]:
max(np.sum(np.array(bus_prop_mat_prop), axis=1))

1.0000000000000004

In [251]:
avg_carbon_emission_rate_node = carbon_emission @ bus_prop_mat_prop

In [252]:
df_ = pd.read_csv("GIS/CATS_buses.csv")
df_.insert(2, 'power_demand', power_demand)
df_.insert(3, 'avg carbon emission', avg_carbon_emission_rate_node)
df_.to_csv('GIS/CATS_buses_output.csv', index=False)